In [1]:
!pip -q install pandas requests pyarrow

import os
import base64
import json
import requests
import pandas as pd
from pathlib import Path

# If you skipped Drive above, set SAVE_DIR to a local folder
SAVE_DIR = globals().get('SAVE_DIR', '/content/udl_data')
Path(SAVE_DIR).mkdir(parents=True, exist_ok=True)


In [2]:
import requests, pandas as pd, getpass, os

URL = "https://unifieddatalibrary.com/udl/statevector/tuple?columns=epoch,source,origObjectId,referenceFrame,zpos,ypos,xpos,xvel,yvel,zvel&epoch=2025-08-15T00:00:00.000000Z..2025-08-30T00:00:00.000000Z&maxResults=3000"

def try_basic_auth():
    print("Trying Basic auth...")
    user = getpass.getpass("UDL username: ")
    pw   = getpass.getpass("UDL password: ")
    r = requests.get(URL, auth=(user, pw), headers={"accept":"application/json"}, timeout=120)
    print("Status:", r.status_code, "| WWW-Authenticate:", r.headers.get("www-authenticate","<none>"))
    return r

def try_cookie_auth():
    print("\nTrying Cookie header from browser session...")
    cookie = input("Paste Cookie header value (from DevTools > Network > Request Headers): ").strip()
    r = requests.get(URL, headers={"accept":"application/json","Cookie":cookie}, timeout=120)
    print("Status:", r.status_code)
    return r

# 1) Basic
resp = try_basic_auth()
if resp.status_code == 401:
    # 2) Cookie
    resp = try_cookie_auth()

resp.raise_for_status()
data = resp.json()
df = pd.DataFrame(data)
if "epoch" in df.columns:
    df["epoch"] = pd.to_datetime(df["epoch"], utc=True, errors="coerce")
print("Rows:", len(df))
df.head()


Trying Basic auth...
UDL username: ··········
UDL password: ··········
Status: 200 | WWW-Authenticate: <none>
Rows: 3000


,classificationMarking,epoch,source,origObjectId,referenceFrame,zpos,ypos,xpos,xvel,yvel,zvel
0,U,2025-08-30 00:00:00+00:00,MITRE,None,ECR/ECEF,-21475.865521,6541.223347,-13638.004523,-1.039110,-2.615369,-0.103848
1,U,2025-08-30 00:00:00+00:00,MITRE,None,ECR/ECEF,-14757.024327,12919.226972,18022.997992,0.762328,1.710962,2.395251
2,U,2025-08-30 00:00:00+00:00,MITRE,None,ECR/ECEF,15385.281234,28746.750565,-29338.212017,0.593099,0.085785,1.498081
3,U,2025-08-30 00:00:00+00:00,MITRE,None,ECR/ECEF,-20733.424137,14874.876077,8857.167058,-2.513939,0.208564,-0.908746
4,U,2025-08-30 00:00:00+00:00,MITRE,None,ECR/ECEF,7644.353503,19328.608238,16570.217343,0.224213,1.031004,-2.926396


In [3]:
df = pd.DataFrame(data)

# Basic normalization
if 'epoch' in df.columns:
    df['epoch'] = pd.to_datetime(df['epoch'], utc=True, errors='coerce')

# Reorder typical columns if present
cols = [c for c in ['epoch','source','origObjectId','xpos','ypos','zpos','xvel','yvel','zvel'] if c in df.columns]
df = df[cols + [c for c in df.columns if c not in cols]]

csv_path = f'{SAVE_DIR}/udl_statevectors.csv'
parquet_path = f'{SAVE_DIR}/udl_statevectors.parquet'

df.to_csv(csv_path, index=False)
df.to_parquet(parquet_path, index=False)

csv_path, parquet_path, df.shape


('/content/udl_data/udl_statevectors.csv',
 '/content/udl_data/udl_statevectors.parquet',
 (3000, 11))

In [4]:
!pip -q install pandas pyarrow astropy


In [5]:
import pandas as pd
from pathlib import Path

clean = df.copy()

# Normalize epoch, coerce bad timestamps to NaT
clean["epoch"] = pd.to_datetime(clean["epoch"], utc=True, errors="coerce")

# Standardize column names
rename_map = {
    "origObjectId": "object_id",
    "xpos": "x_ecef_km",
    "ypos": "y_ecef_km",
    "zpos": "z_ecef_km",
    "xvel": "vx_ecef_km_s",
    "yvel": "vy_ecef_km_s",
    "zvel": "vz_ecef_km_s",
    "source": "source_system",
    "referenceFrame": "reference_frame"
}
clean = clean.rename(columns=rename_map)

# Keep only useful cols for now
wanted = ["epoch","object_id","reference_frame",
          "x_ecef_km","y_ecef_km","z_ecef_km",
          "vx_ecef_km_s","vy_ecef_km_s","vz_ecef_km_s",
          "source_system"]
clean = clean[[c for c in wanted if c in clean.columns]]

# Drop rows with missing epoch or vectors
vec_cols = [c for c in clean.columns if c.endswith("_km") or c.endswith("_km_s")]
clean = clean.dropna(subset=["epoch"] + vec_cols)

print(clean.shape)
clean.head()


(3000, 10)


,epoch,object_id,reference_frame,x_ecef_km,y_ecef_km,z_ecef_km,vx_ecef_km_s,vy_ecef_km_s,vz_ecef_km_s,source_system
0,2025-08-30 00:00:00+00:00,None,ECR/ECEF,-13638.004523,6541.223347,-21475.865521,-1.039110,-2.615369,-0.103848,MITRE
1,2025-08-30 00:00:00+00:00,None,ECR/ECEF,18022.997992,12919.226972,-14757.024327,0.762328,1.710962,2.395251,MITRE
2,2025-08-30 00:00:00+00:00,None,ECR/ECEF,-29338.212017,28746.750565,15385.281234,0.593099,0.085785,1.498081,MITRE
3,2025-08-30 00:00:00+00:00,None,ECR/ECEF,8857.167058,14874.876077,-20733.424137,-2.513939,0.208564,-0.908746,MITRE
4,2025-08-30 00:00:00+00:00,None,ECR/ECEF,16570.217343,19328.608238,7644.353503,0.224213,1.031004,-2.926396,MITRE


In [ ]:

import numpy as np
from astropy.time import Time
from astropy import units as u
from astropy.coordinates import (
    CartesianRepresentation, CartesianDifferential,
    ITRS, GCRS
)

eci = clean.copy()



# Which rows are Earth fixed
mask_ecef = eci["reference_frame"].astype(str).str.contains("ECEF|ECR", case=False, na=False)



epochs_sec = eci.loc[mask_ecef, "epoch"].astype("int64")/1e9  
t = Time(epochs_sec.to_numpy(), format="unix", scale="utc")

# 3) Positions and velocities in ECEF
pos = CartesianRepresentation(
    eci.loc[mask_ecef, "x_ecef_km"].to_numpy()*u.km,
    eci.loc[mask_ecef, "y_ecef_km"].to_numpy()*u.km,
    eci.loc[mask_ecef, "z_ecef_km"].to_numpy()*u.km,
)
vel = CartesianDifferential(
    eci.loc[mask_ecef, "vx_ecef_km_s"].to_numpy()*u.km/u.s,
    eci.loc[mask_ecef, "vy_ecef_km_s"].to_numpy()*u.km/u.s,
    eci.loc[mask_ecef, "vz_ecef_km_s"].to_numpy()*u.km/u.s,
)

# 4) Transform ITRS(ECEF) -> GCRS(ECI-like)
itrs = ITRS(pos.with_differentials(vel), obstime=t)
gcrs = itrs.transform_to(GCRS(obstime=t))

# 5) Write ECI columns back
eci.loc[mask_ecef, "x_eci_km"]  = gcrs.cartesian.x.to_value(u.km)
eci.loc[mask_ecef, "y_eci_km"]  = gcrs.cartesian.y.to_value(u.km)
eci.loc[mask_ecef, "z_eci_km"]  = gcrs.cartesian.z.to_value(u.km)
d = gcrs.cartesian.differentials["s"]
eci.loc[mask_ecef, "vx_eci_km_s"] = d.d_x.to_value(u.km/u.s)
eci.loc[mask_ecef, "vy_eci_km_s"] = d.d_y.to_value(u.km/u.s)
eci.loc[mask_ecef, "vz_eci_km_s"] = d.d_z.to_value(u.km/u.s)

# 6) Mark standardized frame and sort
eci["frame_standard"] = "ECI"
eci = eci.sort_values(["epoch"]).reset_index(drop=True)

eci.head()


,epoch,object_id,reference_frame,x_ecef_km,y_ecef_km,z_ecef_km,vx_ecef_km_s,vy_ecef_km_s,vz_ecef_km_s,source_system,x_eci_km,y_eci_km,z_eci_km,vx_eci_km_s,vy_eci_km_s,vz_eci_km_s,frame_standard
0,2025-08-29 21:50:00+00:00,None,ECR/ECEF,-7810.993866,17604.102343,-16726.759547,-1.597713,1.657271,2.477772,MITRE,9747.327355,16585.096503,-16751.856997,-0.782751,2.977213,2.479620,ECI
1,2025-08-29 21:50:00+00:00,None,ECR/ECEF,-12492.572639,18501.319614,-14104.808829,-0.060367,-1.838901,-2.373504,MITRE,7763.973465,20916.759797,-14125.109073,-3.062963,-0.450692,-2.365831,ECI
2,2025-08-29 21:50:00+00:00,None,ECR/ECEF,13374.617289,9780.790895,-21317.729722,-2.117764,1.489837,-0.652992,MITRE,15678.099603,-5203.241633,-21356.809875,0.359793,3.736380,-0.654040,ECI
3,2025-08-29 21:50:00+00:00,None,ECR/ECEF,-1653.375021,-24866.449980,-8887.242690,0.487629,-1.018484,2.921045,MITRE,-21221.154019,-13102.905137,-8833.660922,0.417164,-2.534382,2.920109,ECI
4,2025-08-29 21:50:00+00:00,None,ECR/ECEF,-21856.968852,13715.245834,7183.432211,-0.882706,0.082784,-2.988041,MITRE,-1519.049220,25758.248500,7186.228337,-2.331273,0.654318,-2.982246,ECI


In [7]:
out_dir = Path("/content/udl_data")
out_dir.mkdir(parents=True, exist_ok=True)
eci.to_parquet(out_dir/"state_vectors_eci.parquet", index=False)
eci.to_csv(out_dir/"state_vectors_eci.csv", index=False)
print("Saved:", out_dir)


Saved: /content/udl_data


# NOW, Extrctaing  emphemeris datasets.


In [8]:
import requests, pandas as pd, getpass, os
from pathlib import Path


URL = "https://unifieddatalibrary.com/udl/ephemerisset/tuple?columns=createdAt,id,origObjectId,pointEndTime,pointStartTime,referenceFrame,source,stepSize,idOnOrbit,numPoints&pointEndTime=%3E2025-08-15T00:00:00.000000Z&pointStartTime=%3C2025-08-30T00:00:00.000000Z&maxResults=3000"

user = getpass.getpass("UDL username: ")
pw   = getpass.getpass("UDL password: ")

resp = requests.get(URL, auth=(user, pw), headers={"accept": "application/json"}, timeout=180)
print("HTTP status:", resp.status_code)
resp.raise_for_status()

sets = pd.DataFrame(resp.json())

# Parse times to UTC datetimes for easy filtering later
for col in ["createdAt", "pointStartTime", "pointEndTime"]:
    if col in sets.columns:
        sets[col] = pd.to_datetime(sets[col], utc=True, errors="coerce")

# Save
out_dir = Path("/content/udl_data"); out_dir.mkdir(parents=True, exist_ok=True)
csv_path = out_dir / "ephemeris_sets.csv"
parq_path = out_dir / "ephemeris_sets.parquet"

sets.to_csv(csv_path, index=False)
sets.to_parquet(parq_path, index=False)

print(f"Saved {len(sets)} rows")
sets.head()


UDL username: ··········
UDL password: ··········
HTTP status: 200
Saved 227 rows


,classificationMarking,createdAt,id,origObjectId,pointEndTime,pointStartTime,referenceFrame,source,stepSize,idOnOrbit,numPoints
0,U,2025-08-16 18:52:25.149000+00:00,976b0dab-713d-4696-947e-585686ba34da,None,2025-08-15 19:52:00+00:00,2025-08-09 00:00:00+00:00,ITRF,saber,None,53105,4917
1,U,2025-08-17 18:53:16.101000+00:00,57f03102-f98b-43ad-93cb-3482c2ffd27f,None,2025-08-16 19:20:00+00:00,2025-08-10 00:00:00+00:00,ITRF,saber,None,53105,4901
2,U,2025-08-18 18:54:17.402000+00:00,ddd2d780-1361-4a33-bf8b-9081c68fb5d2,None,2025-08-17 20:36:00+00:00,2025-08-11 00:00:00+00:00,ITRF,saber,None,53105,4939
3,U,2025-08-16 18:52:22.201000+00:00,8ba6cc83-8ee2-4f7d-97ff-fd67d03c73d1,None,2025-08-15 19:54:00+00:00,2025-08-12 00:00:00+00:00,ITRF,saber,None,38077,1839
4,U,2025-08-16 18:52:20.192000+00:00,b8283cbd-0d9b-47b9-a424-8063a9b65961,None,2025-08-15 20:30:00+00:00,2025-08-12 00:00:00+00:00,ITRF,saber,None,27944,1851


In [9]:
# sets is your DataFrame from the EphemerisSet call
print("Sets returned:", len(sets))
if "numPoints" in sets.columns:
    print("Total points across sets:", sets["numPoints"].fillna(0).sum())

# See frames and date coverage
print(sets["referenceFrame"].value_counts(dropna=False))
print(sets[["pointStartTime","pointEndTime"]].min(), sets[["pointStartTime","pointEndTime"]].max())


Sets returned: 227
Total points across sets: 523060
referenceFrame
ITRF    227
Name: count, dtype: int64
pointStartTime   2025-08-09 00:00:00+00:00
pointEndTime     2025-08-15 19:52:00+00:00
dtype: datetime64[ns, UTC] pointStartTime   2025-08-28 00:00:00+00:00
pointEndTime     2025-08-31 22:52:00+00:00
dtype: datetime64[ns, UTC]


In [16]:
import requests, pandas as pd
import getpass

USER = input("UDL username: ")
PW   = getpass.getpass("UDL password: ")

esid = "976b0dab-713d-4696-947e-585686ba34da"

url = "https://unifieddatalibrary.com/udl/ephemeris/tuple"

params = {
    "columns": "esId,id,xpos,ypos,zpos,xvel,yvel,zvel,source",
    "esId": esid,
    "maxResults": 500
}

resp = requests.get(url, auth=(USER, PW), headers={"accept":"application/json"}, params=params, timeout=120)

print("HTTP status:", resp.status_code)
print("Final URL:", resp.url)

if resp.status_code == 200:
    df = pd.DataFrame(resp.json())
    print("Rows returned:", len(df))
    display(df.head())
else:
    print("Error response:", resp.text[:500])


UDL username: kamayani.rai
UDL password: ··········
HTTP status: 200
Final URL: https://unifieddatalibrary.com/udl/ephemeris/tuple?columns=esId%2Cid%2Cxpos%2Cypos%2Czpos%2Cxvel%2Cyvel%2Czvel%2Csource&esId=976b0dab-713d-4696-947e-585686ba34da&maxResults=500
Rows returned: 500


,classificationMarking,esId,id,xpos,ypos,zpos,xvel,yvel,zvel,source
0,U,976b0dab-713d-4696-947e-585686ba34da,a81cf71b-1b1f-4d58-9ee3-fa59d68bd161,1870.115666,-11870.311882,-2507.378320,0.916629,1.245079,-5.231925,saber
1,U,976b0dab-713d-4696-947e-585686ba34da,81914156-1389-4bf3-a051-f45780d795fc,1978.631651,-11703.968318,-3130.990662,0.892337,1.526676,-5.158925,saber
2,U,976b0dab-713d-4696-947e-585686ba34da,b5da20ec-1274-487f-8b69-2e3984d28427,2084.364628,-11504.080275,-3744.880002,0.870245,1.804006,-5.069918,saber
3,U,976b0dab-713d-4696-947e-585686ba34da,83c5cb87-7c87-49eb-a4e0-0a13ef91e561,2187.577903,-11271.212565,-4347.141993,0.850338,2.076193,-4.965187,saber
4,U,976b0dab-713d-4696-947e-585686ba34da,b4131274-fe78-4414-807e-42e2becdac35,2288.531733,-11006.034186,-4935.909078,0.832580,2.342376,-4.845062,saber


In [17]:
import requests
import pandas as pd
from urllib.parse import urlencode
from pathlib import Path
import getpass
import time

# --- credentials ---
USER = input("UDL username: ")
PW   = getpass.getpass("UDL password: ")

# --- load sets metadata ---
sets = pd.read_csv("udl_data/ephemeris_sets.csv")
esids = sets["id"].dropna().unique().tolist()
print("Total EphemerisSets to fetch:", len(esids))

# --- UDL endpoint for ephemeris points ---
url = "https://unifieddatalibrary.com/udl/ephemeris/tuple"

all_points = []
success, skipped = 0, 0

for esid in esids:
    params = {
        "columns": "esId,id,xpos,ypos,zpos,xvel,yvel,zvel,source",
        "esId": esid,
        "maxResults": 20000  # adjust if needed
    }

    try:
        resp = requests.get(url, auth=(USER, PW),
                            headers={"accept":"application/json"},
                            params=params, timeout=120)
        if resp.status_code != 200:
            print(f"⚠️ Skip esId {esid}: HTTP {resp.status_code}")
            skipped += 1
            continue

        df = pd.DataFrame(resp.json())
        if df.empty:
            print(f"⚠️ No points for esId {esid}")
            skipped += 1
            continue

        df["esId"] = esid
        all_points.append(df)
        success += 1
        print(f"✓ esId {esid}: {len(df)} points")

        time.sleep(1)  # throttle

    except Exception as e:
        print(f"⚠️ Error fetching {esid}: {e}")
        skipped += 1
        continue

# --- combine all results ---
if all_points:
    big_df = pd.concat(all_points, ignore_index=True)
    print("\n✅ Final dataset size:", len(big_df))
    print(f"✓ Success: {success} sets | ⚠️ Skipped: {skipped} sets")

    out_dir = Path("udl_data"); out_dir.mkdir(parents=True, exist_ok=True)
    big_df.to_csv(out_dir/"ephemeris_points_raw.csv", index=False)
    big_df.to_parquet(out_dir/"ephemeris_points_raw.parquet", index=False)

    print(f"💾 Saved to {out_dir/'ephemeris_points_raw.csv'} and .parquet")
else:
    print("❌ No ephemeris points were retrieved.")


UDL username: kamayani.rai
UDL password: ··········
Total EphemerisSets to fetch: 227
✓ esId 976b0dab-713d-4696-947e-585686ba34da: 4917 points
✓ esId 57f03102-f98b-43ad-93cb-3482c2ffd27f: 4901 points
✓ esId ddd2d780-1361-4a33-bf8b-9081c68fb5d2: 4939 points
✓ esId 8ba6cc83-8ee2-4f7d-97ff-fd67d03c73d1: 1839 points
✓ esId b8283cbd-0d9b-47b9-a424-8063a9b65961: 1851 points
✓ esId a57c13a9-4637-4eb8-8c3b-b948b690f54f: 1856 points
✓ esId 0ec95ace-5771-4dcc-89dd-e1d4a1c2899f: 1399 points
✓ esId aad6eb98-ec13-45e8-bca7-f6b1522ace37: 1904 points
✓ esId d653a9da-483b-420f-89e8-37d6054f37f9: 5011 points
✓ esId 1cf0035a-8a5e-4aa0-8394-259029027e7e: 5011 points
✓ esId 23220c65-f95f-406d-b276-962ff12a2ee6: 5011 points
✓ esId 7c521e6d-73ca-4052-bf5b-dba254b2857c: 5011 points
✓ esId 842bc3ee-1563-4530-a125-badcee1732d8: 5011 points
✓ esId e69d5ffb-1e79-4bf7-95e9-00b70a1377d2: 5011 points
✓ esId 333e6966-8295-4ec9-8371-617927423abb: 5011 points
✓ esId 25043549-ad3a-4584-bc09-aa8977cd3558: 5011 points
✓ 

In [18]:
import requests
import pandas as pd
from urllib.parse import urlencode
from pathlib import Path
import getpass
import time

# --- credentials ---
USER = input("UDL username: ")
PW   = getpass.getpass("UDL password: ")

# --- load sets metadata ---
sets = pd.read_csv("udl_data/ephemeris_sets.csv")
esids = sets["id"].dropna().unique().tolist()
print("Total EphemerisSets to fetch:", len(esids))

# --- UDL endpoint for ephemeris points ---
url = "https://unifieddatalibrary.com/udl/ephemeris/tuple"

all_points = []
success, skipped = 0, 0

for esid in esids:
    params = {
        "columns": "esId,id,xpos,ypos,zpos,xvel,yvel,zvel,source",
        "esId": esid,
        "maxResults": 20000
    }

    try:
        resp = requests.get(
            url, auth=(USER, PW),
            headers={"accept":"application/json"},
            params=params, timeout=120
        )
        if resp.status_code != 200:
            print(f"⚠️ Skip esId {esid}: HTTP {resp.status_code}")
            skipped += 1
            continue

        df = pd.DataFrame(resp.json())
        if df.empty:
            print(f"⚠️ No points for esId {esid}")
            skipped += 1
            continue

        df["esId"] = esid
        all_points.append(df)
        success += 1
        print(f"✓ esId {esid}: {len(df)} points")

        time.sleep(0.5)  # gentle throttle

    except Exception as e:
        print(f"⚠️ Error fetching {esid}: {e}")
        skipped += 1
        continue

# --- combine all results ---
if all_points:
    big_df = pd.concat(all_points, ignore_index=True)
    print("\n✅ Raw dataset size:", len(big_df))
    print(f"✓ Success: {success} sets | ⚠️ Skipped: {skipped} sets")

    # --- enrich with metadata ---
    merged = big_df.merge(
        sets[["id","pointStartTime","pointEndTime","referenceFrame","source"]],
        left_on="esId", right_on="id", how="left"
    ).drop(columns=["id_y"]).rename(columns={"id_x":"pointId","id":"esId"})

    # --- generate approximate epochs ---
    merged["epoch"] = pd.NaT
    for esid, group in merged.groupby("esId"):
        try:
            start = pd.to_datetime(group["pointStartTime"].iloc[0], utc=True, errors="coerce")
            end   = pd.to_datetime(group["pointEndTime"].iloc[0], utc=True, errors="coerce")
            n     = len(group)
            if pd.notna(start) and pd.notna(end) and n > 1:
                times = pd.date_range(start, end, periods=n)
                merged.loc[group.index, "epoch"] = times
        except Exception as e:
            print(f"⚠️ Could not generate epochs for {esid}: {e}")

    # --- clean column names ---
    merged = merged.rename(columns={
        "xpos": "x_km",
        "ypos": "y_km",
        "zpos": "z_km",
        "xvel": "vx_kms",
        "yvel": "vy_kms",
        "zvel": "vz_kms"
    })

    # --- save cleaned data ---
    out_dir = Path("udl_data"); out_dir.mkdir(parents=True, exist_ok=True)
    merged.to_csv(out_dir/"ephemeris_points_clean.csv", index=False)
    merged.to_parquet(out_dir/"ephemeris_points_clean.parquet", index=False)

    print(f"\n💾 Saved cleaned ephemeris points to:\n  {out_dir/'ephemeris_points_clean.csv'}")
else:
    print("❌ No ephemeris points were retrieved.")


UDL username: kamayani.rai
UDL password: ··········
Total EphemerisSets to fetch: 227
✓ esId 976b0dab-713d-4696-947e-585686ba34da: 4917 points
✓ esId 57f03102-f98b-43ad-93cb-3482c2ffd27f: 4901 points
✓ esId ddd2d780-1361-4a33-bf8b-9081c68fb5d2: 4939 points
✓ esId 8ba6cc83-8ee2-4f7d-97ff-fd67d03c73d1: 1839 points
✓ esId b8283cbd-0d9b-47b9-a424-8063a9b65961: 1851 points
✓ esId a57c13a9-4637-4eb8-8c3b-b948b690f54f: 1856 points
✓ esId 0ec95ace-5771-4dcc-89dd-e1d4a1c2899f: 1399 points
✓ esId aad6eb98-ec13-45e8-bca7-f6b1522ace37: 1904 points
✓ esId d653a9da-483b-420f-89e8-37d6054f37f9: 5011 points
✓ esId 1cf0035a-8a5e-4aa0-8394-259029027e7e: 5011 points
✓ esId 23220c65-f95f-406d-b276-962ff12a2ee6: 5011 points
✓ esId 7c521e6d-73ca-4052-bf5b-dba254b2857c: 5011 points
✓ esId 842bc3ee-1563-4530-a125-badcee1732d8: 5011 points
✓ esId e69d5ffb-1e79-4bf7-95e9-00b70a1377d2: 5011 points
✓ esId 333e6966-8295-4ec9-8371-617927423abb: 5011 points
✓ esId 25043549-ad3a-4584-bc09-aa8977cd3558: 5011 points
✓ 

/tmp/ipython-input-834854422.py:80: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'DatetimeIndex(['2025-08-19 00:00:00+00:00', '2025-08-19 00:02:00+00:00',
               '2025-08-19 00:04:00+00:00', '2025-08-19 00:06:00+00:00',
               '2025-08-19 00:08:00+00:00', '2025-08-19 00:10:00+00:00',
               '2025-08-19 00:12:00+00:00', '2025-08-19 00:14:00+00:00',
               '2025-08-19 00:16:00+00:00', '2025-08-19 00:18:00+00:00',
               ...
               '2025-08-25 19:58:00+00:00', '2025-08-25 20:00:00+00:00',
               '2025-08-25 20:02:00+00:00', '2025-08-25 20:04:00+00:00',
               '2025-08-25 20:06:00+00:00', '2025-08-25 20:08:00+00:00',
               '2025-08-25 20:10:00+00:00', '2025-08-25 20:12:00+00:00',
               '2025-08-25 20:14:00+00:00', '2025-08-25 20:16:00+00:00'],
              dtype='datetime64[ns, UTC]', length=4929, freq=None)' has dtype incompa


💾 Saved cleaned ephemeris points to:
  udl_data/ephemeris_points_clean.csv


# Extrating dataset object of interest.

In [21]:
import requests, base64, getpass, pandas as pd
from pathlib import Path

# --- credentials ---
USER = input("UDL username: ")
PW   = getpass.getpass("UDL password: ")

# --- build Basic Auth header ---
key = f"{USER}:{PW}"
token = base64.b64encode(key.encode("utf-8")).decode("ascii")
headers = {
    "accept": "application/json",
    "Authorization": f"Basic {token}"
}

# --- endpoint ---
url = "https://unifieddatalibrary.com/udl/objectofinterest/tuple"
params = {
    "columns": "createdAt,id,name,origin,classificationMarking,updatedAt,description,source,status,idOnOrbit,elsetEpoch,svEpoch",
    "maxResults": 3000
}

# --- request ---
resp = requests.get(url, headers=headers, params=params, timeout=180)

print("HTTP status:", resp.status_code)
print("Final URL:", resp.url)

if resp.status_code == 200:
    df = pd.DataFrame(resp.json())
    print("Rows retrieved:", len(df))
    display(df.head())

    out = Path("udl_data"); out.mkdir(exist_ok=True)
    df.to_csv(out/"object_of_interest.csv", index=False)
    df.to_parquet(out/"object_of_interest.parquet", index=False)
    print(f"Saved to {out/'object_of_interest.csv'} and .parquet")
else:
    print("Error response:", resp.text[:500])


UDL username: kamayani.rai
UDL password: ··········
HTTP status: 200
Final URL: https://unifieddatalibrary.com/udl/objectofinterest/tuple?columns=createdAt%2Cid%2Cname%2Corigin%2CclassificationMarking%2CupdatedAt%2Cdescription%2Csource%2Cstatus%2CidOnOrbit%2CelsetEpoch%2CsvEpoch&maxResults=3000
Rows retrieved: 1061


,createdAt,id,name,origin,classificationMarking,updatedAt,description,source,status,idOnOrbit,elsetEpoch,svEpoch
0,2019-08-29T19:30:42.625Z,F00F93CAD81B4703BB9EF88308AC56AF,ARABSAT 5C,None,U,None,None,Odin,OPEN,37810,2019-08-28T07:30:46.999000Z,None
1,2019-08-29T20:19:31.507Z,3A4DA2C7579A4BBFB83A6746AD26C937,WORLDVIEW-4,None,U,None,None,Odin,OPEN,41848,2019-08-28T04:19:53.000000Z,None
2,2019-09-04T21:32:14.503Z,12d48dc6-98da-4451-be40-b8f694bb35dd,unknown,None,U,None,manually generated by SensorIQ,SensorIQ,OPEN,27445,None,None
3,2019-09-04T21:36:44.266Z,275f6428-cf6d-43c7-a3f2-16d2caec2dc1,unknown,None,U,None,manually generated by SensorIQ,SensorIQ,OPEN,27854,None,None
4,2019-09-06T18:29:11.750Z,36c1c790-ff67-4610-978d-350f4cc38232,unknown,None,U,None,manually generated by SensorIQ,SensorIQ,OPEN,37810,None,None


Saved to udl_data/object_of_interest.csv and .parquet
